### [CNN 구조]

- torch.nn 패키지를 사용하여 신경망을 생성함.
- nn 패키지는 모델을 정의할 때, autograd를 통해 자동미분 기능을 제공함
- nn.Module은 층과 전방전파forward propagation (입력à출력)을 쉽게 구현함
- nn패키지는 mini-batch만 지원함, 예로nn.Covn2d는 4차Tensor를 받음(nSamples * nChannels * height * width)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

***
**출력 확인**

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


***
**정의된 합성곱 신경망 구조 설명**

> 1. 첫 번째 conv층('self.conv1')

- 입력 : 입력 x를 받는다.(ex : 이미지)
- conv연산 : input 채널 1, output 채널 6, 5x5 conv -> 공간적 특징 추출
- ReLU함수 : 비선형성 도입
- MaxPooling : 2x2 window를 사용하여 각 윈도우 내 최대값 선택 -> 공간적 특징을 줄이고, 중요한 특징을 강조


> 2. 두 번째 conv층('self.conv2')

- conv연산 : input 6, output 16, 5x5 conv
- ReLU & MaxPooling


> 3. Flatten

- 변환 : 'x = x.view(-1, self.num_flat_features(x))'를 통해 conv층의 2차원 출력을 1차원 벡터로 반환 -> view(-1, ...)는 데이터를 1차원으로 재구성하는데 사용되며, '-1'은 해당 차원의 크기를 자동으로 계산하게 함


> 4. FC layer(완전 연결 층)

- fc1, fc2, fc3 세 번의 완전 연결 층을 거치며 최종적으로 분류 문제에 사용될 클래스에 대한 점수나 확률을 나타내는 값을 반환한다.

***

**신경망의 학습가능한 parameters 출력**

In [4]:
# net.parameters()를 사용하여 정의된 신경망의 학습가능한 매개변수들을 확인할 수 있음
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


***
**임의의 32*32 입력을 가정 후 출력 확인**

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0905, -0.1263, -0.0690,  0.0321, -0.0324, -0.0201, -0.0709, -0.0026,
          0.0980, -0.0932]], grad_fn=<AddmmBackward0>)


***

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

# 손실 함수 정의 및 임의의 값들에 대해서 오차 결과 확인
# nn 패키지는 많이 사용되는 손실함수들을 제공하며, 해당 예제는 단순한 MSE를 사용
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2831, grad_fn=<MseLossBackward0>)


In [7]:
# 앞에 코드에서 언급한 것과 같이 오류 역전파하기 전, 그레이디언트를 초기화해야 함
# backward() 수행 후 어떤 변화가 있는지 확인하고, 초기화의 필요성을 확인함
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None


In [8]:
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0021,  0.0159,  0.0135,  0.0131, -0.0047,  0.0064])


In [10]:
# 스토캐스틱경사하강법( (미래)가중치= (현재)가중치–학습률*그레이디언트)을이용하여가중치갱신하는코드는다음과같음
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)